In [ ]:
pip install beautifulsoup4

In [ ]:
urls = pd.read_csv('/content/ontario_links.csv',header=None)
url_list = urls[0].tolist()
url_list

In [ ]:
# Check names

import requests
from bs4 import BeautifulSoup

# URL of the law page
url = "https://reg.pravno-informacioni-sistem.rs/api/viewdoc?uuid=188df76e-84d1-47d8-8653-a23b16503c04&regactid=433539&doctype=reg"

# Send a GET request to fetch the webpage
response = requests.get(url, verify=False)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract document name
    document_name_tag = soup.find('h2', class_='Title-of-Act')
    if document_name_tag:
        document_name = document_name_tag.get_text(strip=True)
        print(f"Document Name: {document_name}")
    else:
        print("Document name not found")

    # Now let's print out a portion of the HTML to inspect the structure around the sections
    # Printing the first 1000 characters of the content for inspection
    print("\nRaw HTML Sample:\n")
    print(soup.prettify()[2000:19000])  # Print first 1000 characters of HTML

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Lista de URLs a scrape
urls = pd.read_csv('/content/serbia_links.csv')['link']
urls = urls.tolist()
urls


# Inicializa un DataFrame para almacenar los datos concatenados
all_data = pd.DataFrame()

# Función para scrape una URL
def scrape_url(url):
    try:
        response = requests.get(url,verify=False)
        if response.status_code == 200:
            # Parsear el contenido HTML de la página
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraer el nombre del documento de <p class="odluka-zakon">
            document_name_tags = soup.find_all('p', class_='odluka-zakon')
            document_name = " ".join([tag.get_text(strip=True) for tag in document_name_tags]) if document_name_tags else "Desconocido"

            # Inicializa una lista para almacenar las filas de esta URL
            data = []

            # Encuentra todas las secciones de título (p class="clan", incluyendo las que tienen id)
            section_titles = soup.find_all('p', class_='clan')

            for section_title in section_titles:
                # Extraer el texto del título actual
                current_section_title = section_title.get_text(strip=True)

                # Inicializa una lista para almacenar el texto de la sección
                current_text = []

                # Recorre los elementos siguientes hasta encontrar el siguiente document_section_title
                for sibling in section_title.find_next_siblings():
                    # Detenerse al encontrar otro document_section_title con id
                    if sibling.name == 'p' and 'clan' in sibling.get('class', []) and sibling.get('id'):
                        break
                    if sibling.name == 'p' and 'clan' in sibling.get('class', []):
                        break  # Detenerse al encontrar otro document_section_title
                    if sibling.name == 'p' and 'bold' in sibling.get('class', []):
                        continue  # Saltar los <p> con clase 'bold'
                    if sibling.name == 'p' and any(cls in sibling.get('class', []) for cls in ['v2-clan-left-1', 'v2-clan-left-2', 'v2-clan-left-3', 'v2-clan-left-4', 'v2-clan-left-5']):
                        current_text.append(sibling.get_text(" ", strip=True))
                    elif sibling.name == 'p':
                        current_text.append(sibling.get_text(" ", strip=True))

                # Guarda el título de la sección con su texto correspondiente
                row = {
                    'document_name': document_name,
                    'document_section_title': current_section_title,
                    'text': ' '.join(current_text).strip()
                }
                data.append(row)

            # Convierte los datos scrapeados en un DataFrame
            df = pd.DataFrame(data)
            return df

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return pd.DataFrame()

# Loop a través de cada URL con un retardo entre las solicitudes
for url in urls:
    print(f"Scraping: {url}")
    df = scrape_url(url)

    if not df.empty:
        # Concatenar los nuevos datos con el DataFrame existente
        all_data = pd.concat([all_data, df], ignore_index=True)

    # Pausa entre solicitudes para evitar sobrecargar el servidor
    time.sleep(5)  # Pausa de 5 segundos (puedes ajustar este valor)

# Guarda el DataFrame final concatenado en un archivo Excel con codificación UTF-8
all_data.to_excel('serbia_laws.xlsx', index=False)

# Muestra el resultado
print(all_data)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Lista de URLs a scrape
urls = pd.read_csv('/content/serbia_links.csv')['link']
urls = urls.tolist()

# Inicializa un DataFrame para almacenar los datos concatenados
all_data = pd.DataFrame()

# Inicializa una lista para almacenar el nombre del documento y el enlace
document_links = []

# Función para scrape una URL
def scrape_url(url):
    try:
        response = requests.get(url, verify=False)  # Solo usa verify=False si es necesario
        if response.status_code == 200:
            # Parsear el contenido HTML de la página
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraer el nombre del documento de <p class="odluka-zakon">
            document_name_tags = soup.find_all('p', class_='odluka-zakon')
            document_name = " ".join([tag.get_text(strip=True) for tag in document_name_tags]) if document_name_tags else "Desconocido"

            # Añadir el nombre del documento y el enlace a la lista de enlaces
            document_links.append({'document_name': document_name, 'link': url})

            # Inicializa una lista para almacenar las filas de esta URL
            data = []

            # Encuentra todas las secciones de título (p class="clan", incluyendo las que tienen id)
            section_titles = soup.find_all('p', class_='clan')

            for section_title in section_titles:
                # Extraer el texto del título actual
                current_section_title = section_title.get_text(strip=True)

                # Inicializa una lista para almacenar el texto de la sección
                current_text = []

                # Recorre los elementos siguientes hasta encontrar el siguiente document_section_title
                for sibling in section_title.find_next_siblings():
                    # Detenerse al encontrar otro document_section_title con id
                    if sibling.name == 'p' and 'clan' in sibling.get('class', []) and sibling.get('id'):
                        break
                    if sibling.name == 'p' and 'clan' in sibling.get('class', []):
                        break  # Detenerse al encontrar otro document_section_title
                    if sibling.name == 'p' and 'bold' in sibling.get('class', []):
                        continue  # Saltar los <p> con clase 'bold'
                    if sibling.name == 'p' and any(cls in sibling.get('class', []) for cls in ['v2-clan-left-1', 'v2-clan-left-2', 'v2-clan-left-3', 'v2-clan-left-4', 'v2-clan-left-5']):
                        current_text.append(sibling.get_text(" ", strip=True))
                    elif sibling.name == 'p':
                        current_text.append(sibling.get_text(" ", strip=True))

                # Guarda el título de la sección con su texto correspondiente
                row = {
                    'document_name': document_name,
                    'document_section_title': current_section_title,
                    'text': ' '.join(current_text).strip()
                }
                data.append(row)

            # Convierte los datos scrapeados en un DataFrame
            df = pd.DataFrame(data)
            return df

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return pd.DataFrame()

# Loop a través de cada URL con un retardo entre las solicitudes
for url in urls:
    print(f"Scraping: {url}")
    df = scrape_url(url)

    if not df.empty:
        # Concatenar los nuevos datos con el DataFrame existente
        all_data = pd.concat([all_data, df], ignore_index=True)

    # Pausa entre solicitudes para evitar sobrecargar el servidor
    time.sleep(5)  # Pausa de 5 segundos (puedes ajustar este valor)

# Guarda el DataFrame final concatenado en un archivo CSV
all_data.to_csv('laws.csv', index=False)

# Guarda el DataFrame final concatenado en un archivo Excel con codificación UTF-8
all_data.to_excel('laws.xlsx', index=False)

# Crear un DataFrame para el nombre de los documentos y los enlaces
document_links_df = pd.DataFrame(document_links)

# Guardar los nombres de los documentos y enlaces en un archivo Excel
document_links_df.to_excel('document_links.xlsx', index=False)

# Muestra los nombres de los documentos y enlaces
print(document_links_df)


In [ ]:
all_data


,document_name,document_section_title,text
0,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,ПРВИ ДЕО,
1,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,НАЧЕЛА УСТАВА,
2,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,Члан 1.,Република Србија је држава српског народа и св...
3,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,Члан 2.,Сувереност потиче од грађана који је врше рефе...
4,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,Члан 3.,Владавина права је основна претпоставка Устава...
...,...,...,...
7128,Desconocido,Члан 43.*,Инсталације и уређаји за аутоматско откривање ...
7129,Desconocido,Члан 44.*,Исправност инсталација и \r\n\tуређаја за ауто...
7130,Desconocido,Члан 44а*,Министар доноси техничке \r\n\tпрописе којима ...
7131,Desconocido,Члан 38.,Подзаконски прописи \r\n\tпредвиђени овим зако...


In [ ]:
# Eliminar desconocido, eliminar las filas vacias de texto

# Eliminar filas donde el document_name es "Desconocido"
all_data_cleaned = all_data[all_data['document_name'] != 'Desconocido']

# Eliminar filas donde la columna 'text' esté vacía
all_data_cleaned = all_data_cleaned.dropna(subset=['text'])

document_names_links = pd.read_csv('/content/document_names_links.csv')


merged_data = pd.merge(all_data, document_names_links, on='document_name', how='left')

# Guardar la tabla final en un archivo Excel
merged_data.to_excel('serbia_laws.xlsx', index=False)

merged_data


,document_name,document_section_title,text,link,year,basic_services_and_mobility,culture_and_recreation,economy,education,environment,governance,health_and_wellbeing,housing,social_cohesion,context_or_history,population
0,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,ПРВИ ДЕО,,https://reg.pravno-informacioni-sistem.rs/api/...,2022.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,НАЧЕЛА УСТАВА,,https://reg.pravno-informacioni-sistem.rs/api/...,2022.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,Члан 1.,Република Србија је држава српског народа и св...,https://reg.pravno-informacioni-sistem.rs/api/...,2022.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,Члан 2.,Сувереност потиче од грађана који је врше рефе...,https://reg.pravno-informacioni-sistem.rs/api/...,2022.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,УСТАВ РЕПУБЛИКЕ СРБИЈЕ,Члан 3.,Владавина права је основна претпоставка Устава...,https://reg.pravno-informacioni-sistem.rs/api/...,2022.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7128,Desconocido,Члан 43.*,Инсталације и уређаји за аутоматско откривање ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7129,Desconocido,Члан 44.*,Исправност инсталација и \r\n\tуређаја за ауто...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7130,Desconocido,Члан 44а*,Министар доноси техничке \r\n\tпрописе којима ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7131,Desconocido,Члан 38.,Подзаконски прописи \r\n\tпредвиђени овим зако...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pip install fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00
  Created wheel for configobj: filename=configobj-5.0.9-py2.py3-none-any.whl size=35614 sha256=cf6bab4538964b6284eccb7e7aeee2624f6670844f3540a086ac3df42021beb2
  Stored in directory: /root/.cache/pip/wheels/a1/6c/03/6c5e3cf1a6e4b9e

In [ ]:
!sudo apt-get install tesseract-ocr-srp

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-srp
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 780 kB of archives.
After this operation, 2,165 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-srp all 1:4.00~git30-7274cfa-1.1 [780 kB]
Fetched 780 kB in 1s (742 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tesseract-ocr-srp.
(Reading databa

In [ ]:
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import re

# Step 3: Define OCR function using Tesseract
def ocr_image(image):
    return pytesseract.image_to_string(image, lang='srp')  # 'srp' for Serbian language

# Step 4: Convert PDF pages to images
pdf_path = '/content/STATUT - preciscen tekst.pdf'  # Change to the path of your scanned PDF
pages = convert_from_path(pdf_path)

# Function to clean the extracted text by removing specific patterns and leading periods
def clean_text(text):
    # Remove occurrences of '.\n\n' and '.\n'
    cleaned_text = re.sub(r'\.\n+', '. ', text)
    # Remove leading periods at the beginning of the text
    cleaned_text = re.sub(r'^\.\s*', '', cleaned_text)
    return cleaned_text.strip()

# Step 5: Perform OCR on each page and collect text
extracted_text = ""
for page_num, page in enumerate(pages):
    text = ocr_image(page)
    # Clean the text before adding it to the final extracted text
    cleaned_page_text = clean_text(text)
    extracted_text += cleaned_page_text + "\n"

# Step 6: Process text to split into article names and text (based on your pattern 'Члан')
articles = re.split(r'Члан\s*\d+', extracted_text)  # Split based on 'Члан'
article_names = re.findall(r'Члан\s*\d+', extracted_text)

# Step 7: Structure into two columns (Article Name and Text)
data = []
for idx, article in enumerate(articles[1:], start=1):  # Skipping the first item as it is not an article
    article_name = article_names[idx - 1]  # Article name like 'Члан 1'
    article_text = clean_text(article.strip())  # Clean the article text again
    data.append([article_name, article_text])

# Step 8: Create a DataFrame
df = pd.DataFrame(data, columns=['Article Name', 'Text'])

# Step 9: Display DataFrame
df.to_csv('output_STATUT - preciscen teks.csv', index=False)
df


,Article Name,Text
0,Члан 1,"Овим Статутом се уређује положај, статус, подр..."
1,Члан 2,Градска општина Пантелеј је подручје Града Ниш...
2,Члан 3,Градска општина има својство правног лица. Сед...
3,Члан 4,На подручју Градске општине Пантелеј у службен...
4,Члан 5,Градска општина Пантелеј има грб и заставу. Уп...
...,...,...
59,Члан 60,Збор грађана се сазива за део подручја градске...
60,Члан 61,Скупштина Градске општина може на сопствену ин...
61,Члан 62,"Предлог за доношење, односно измену Статута Гр..."
62,Члан 63,Ступањем на снагу овог Статута престаје да важ...


In [ ]:
! pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 66.2 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd

# Step 1: Extract text from the PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

# Step 2: Define the pattern to match article names (e.g., "1. УВОД", "2. Процес израде Плана развоја")
article_pattern = re.compile(r'(\d+(\.\d+)*)\.\s+([^\d\n]+)')

# Step 3: Remove illegal characters
def clean_illegal_characters(text):
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\x00-\x1F\x7F-\x9F]')
    return ILLEGAL_CHARACTERS_RE.sub("", text)

# Step 4: Extract article names and corresponding text
def extract_articles_from_text(text):
    articles = []
    matches = article_pattern.finditer(text)

    # Extract each article with its corresponding text
    last_match_end = 0
    for match in matches:
        if last_match_end > 0:
            articles[-1]["Text"] = clean_illegal_characters(text[last_match_end:match.start()].strip())
        article_name = f"{match.group(1)} {match.group(3)}"
        articles.append({"Article Name": clean_illegal_characters(article_name), "Text": ""})
        last_match_end = match.end()

    # Add the final article's text
    if articles:
        articles[-1]["Text"] = clean_illegal_characters(text[last_match_end:].strip())

    return articles

# Step 5: Process the PDF and extract article names and text
def process_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    articles = extract_articles_from_text(text)
    return articles

# Step 6: Save the results into an Excel file
def save_to_excel(articles, output_path):
    df = pd.DataFrame(articles)
    df.to_excel(output_path, index=False)

# Example usage:
pdf_path = '/content/PlanRazvojaGoPalilula2023-2029.pdf'  # Replace with your PDF file path
output_excel_path = 'extracted_articles.xlsx'  # Path for Excel output

articles = process_pdf(pdf_path)
save_to_excel(articles, output_excel_path)

# Now you can check the "extracted_articles.xlsx" file for the result.


In [ ]:
df

,Article Name,Text
0,Члан 1,"Овим Статутом се уређује положај, статус, подр..."
1,Члан 2,Градска општина Пантелеј је подручје Града Ниш...
2,Члан 3,Градска општина има својство правног лица. Сед...
3,Члан 4,На подручју Градске општине Пантелеј у службен...
4,Члан 5,Градска општина Пантелеј има грб и заставу. Уп...
...,...,...
59,Члан 60,Збор грађана се сазива за део подручја градске...
60,Члан 61,Скупштина Градске општина може на сопствену ин...
61,Члан 62,"Предлог за доношење, односно измену Статута Гр..."
62,Члан 63,Ступањем на снагу овог Статута престаје да важ...
